In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf
import requests
import zipfile
from PIL import Image
from tqdm.notebook import tqdm
import shutil
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import random
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard
import datetime
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pydot
import graphviz
import os
from tqdm.notebook import tqdm

row_data = "./test_data/row_data/"
processed_data = "./test_data/processed_data/"
denoised_data = "./test_data/processed_data/"
classified_data = "./test_data/classified_data/"

image_valid_extensions = [".png", ".jpg", ".jpeg"]
image_target_size = (300, 300)

In [5]:
def process_images(source_directory, destination_directory, target_size, valid_extensions):
    if os.path.exists(destination_directory):
        shutil.rmtree(destination_directory)
    
    # Créer le dossier de destination s'il n'existe pas
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    for file_name in tqdm(os.listdir(source_directory), desc="File", position=1, leave=False):
        if any(file_name.endswith(extension) for extension in valid_extensions):
            source_file_path = os.path.join(source_directory, file_name)
            transient_file_path = os.path.join(destination_directory, "transient_" + file_name)
            destination_file_path = os.path.join(destination_directory, file_name)
            try:
                # Convertir l'image en RGB et redimensionner avec PIL
                with Image.open(source_file_path) as img:
                    img_rgb = img.convert('RGB')
                    img_resized = img_rgb.resize(target_size)
                    img_resized.save(transient_file_path, "JPEG")

                # Lire et vérifier l'image avec TensorFlow
                image = tf.io.read_file(transient_file_path)
                image = tf.image.decode_image(image)

                # Normaliser l'image
                #image = tf.image.convert_image_dtype(image, tf.float32, saturate=True)

                # Enregistrer l'image traitée
                encoded_image = tf.image.encode_jpeg(tf.image.convert_image_dtype(image, dtype=tf.uint8, saturate=True))
                tf.io.write_file(destination_file_path, encoded_image)

            except Exception as e:
                print(f"Erreur lors du traitement de l'image {source_file_path}: {e}")

            if os.path.exists(transient_file_path):
                os.remove(transient_file_path)

    print("Traitement terminé!")
    
process_images(row_data, processed_data, image_target_size, image_valid_extensions)

File:   0%|          | 0/60 [00:00<?, ?it/s]

Traitement terminé!
